In [1]:
import numpy as np
import os
import sys
from iautils import cascade
import pandas as pd
from datetime import datetime
from astropy.coordinates import SkyCoord

/opt/pysetup/.venv/lib/python3.11/site-packages/chime_frb_api/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution as _get_distribution


In [2]:
sys.path.append("/arc/home/mseth/.local/lib/python3.11/site-packages")
import memory_profiler
%load_ext memory_profiler

Filter out all the crab files and save their paths to .npz

In [3]:
source_name = "CYG_A"
coords = SkyCoord.from_name(source_name)
source_ra = coords.ra.deg
source_dec = coords.dec.deg

In [13]:
path = "/arc/projects/chime_frb/adamdong/for_meena"
norescale = 'norescale'
frb = 'frb'

#Files downsampled to 256 frequency channels
filepaths = []
for (root, dirs, file) in os.walk(path):
    for f in file: 
        if norescale in f:
            continue 
        if frb in f:
            continue
        else:
            filepaths.append(os.path.join(root, f))

In [15]:
%%memit
dms = []
crab_filepaths = []

for filepath in filepaths:
    try:
        cascade_obj = cascade.load_cascade_from_file(filepath) 
        dm = cascade_obj.best_snr_dm
        
        if 55 <= dm <= 57:
            dms.append(dm)
            crab_filepaths.append(filepath)
            print(f"{filepath} good")
            del cascade_obj
            continue
        else:
            print(f"{filepath} bad")
            continue 
        
    except Exception as e:
        print(e)
        del cascade_obj
        continue

Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
/arc/projects/chime_frb/adamdong/for_meena/cascade_376412088.npz bad
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
/arc/projects/chime_frb/adamdong/for_meena/cascade_403892485.npz bad
Preprocessing...
using L1 weights
Preprocessin

In [18]:
#Files with full 16384 frequency channel resolution
norescaled_filepaths = []                 
for (root, dirs, file) in os.walk(path):
    for f in file: 
        if norescale in f:
            norescaled_filepaths.append(os.path.join(root, f)) 
        else:
            continue

In [37]:
crabfiles_mjd = []

for crab_filepath in crab_filepaths:
    file = crab_filepath.split("/")
    mjd = file[6].split("_")[1].split(".")[0]
    crabfiles_mjd.append(mjd)

In [43]:
crab_norescaled_filepaths = []

for mjd in crabfiles_mjd:
    for filepath in norescaled_filepaths:
        if mjd in filepath:
            crab_norescaled_filepaths.append(filepath)
        else:
            continue

In [49]:
np.savez("/arc/projects/chime_frb/mseth/Crab_Filepaths", filepath=crab_norescaled_filepaths)

For crab files, pull out all the necessary data and save to a .npz file

In [ ]:
%%memit
dms = []
max_tidxs = []
max_timestamps = []
spectra = []
beam_ids = []
has_at_peak = []
y_at_peak = []
good_filepaths = []

for filepath in crab_norescaled_filepaths:
    try:
        cascade_obj = cascade.load_cascade_from_file(filepath) 
        cascade_obj.dm=57.70 
        dm = cascade_obj.best_snr_dm
        
        if 55 <= dm <= 57:
            ts = np.nansum(cascade_obj.beams[cascade_obj._max_beam_idx].intensity, axis=0)
            max_tidx = np.argmax(ts)
            max_timestamp = cascade_obj.event_time + pd.Timedelta(seconds = max_tidx / 1000)
            spectrum_at_peak = cascade_obj.beams[cascade_obj._max_beam_idx].intensity[:, max_tidx]
            beam_id = cascade_obj.beams[cascade_obj._max_beam_id].beam_no
            
            x, y = utils.get_position_from_equatorial(source_ra, source_dec, max_timestamp)
           
            dms.append(dm)
            max_tidxs.append(max_tidx)
            max_timestamps.append(max_timestamp)
            beam_ids.append(beam_id)
            has_at_peak.append(x)
            y_at_peak.append(y)
            spectra.append(spectrum_at_peak)
            good_filepaths.append(filepath)
            
            print(f"{filepath} good")
            del cascade_obj
        else:
            print(f"{filepath} bad")
            del cascade_obj
            continue
        
    except Exception as e:
        print(e)
        del cascade_obj
        continue

Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersin

In [ ]:
np.savez("Data.npz", DM=dms, MAX_TIDX=max_tidxs, MAX_TIMESTAMP=max_timestamps, BEAM_ID=beam_ids, 
         HA=has_at_peak, Y=y_at_peak)

In [ ]:
spectra_at_peak = np.vstack(spectra)
    np.savez("Spectra.npz", SPECTRUM=spectra_at_peak)